In [36]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob
from datetime import datetime

In [37]:
df = pd.DataFrame()
for files in glob.glob("./data/*.csv"):
    print ('Concatenating...', files)
    df = pd.concat([df,pd.read_csv(files).iloc[:,:]],axis=0,ignore_index=True)
print ('Datasets are now concatenated into one dataframe.')
# Drop first column of dataframe
# df.reset_index(drop=True, inplace=True)


Concatenating... ./data\resale-flat-prices-based-on-approval-date-1990-1999.csv
Concatenating... ./data\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
Concatenating... ./data\resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv
Concatenating... ./data\resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv
Concatenating... ./data\resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv
Datasets are now concatenated into one dataframe.


In [38]:
df.shape

(880072, 11)

In [39]:
df.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
dtype: object

In [40]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


The following column datatypes need to be handled: 
| 'floor_area_sqm' 'resale_price'  'block_num' | to numeric |
| 'month'| to date|

In [41]:

df['block_num'] = df['block'].apply(lambda x: re.findall('\A\d+',x)[0])

In [42]:
# convert numeric columns
for x in ['floor_area_sqm','resale_price', 'block_num']:
    df[x] = pd.to_numeric(df[x])

In [43]:
# change to time format for date columns
df['month'] = pd.to_datetime(df['month'])

In [44]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,block_num
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,309
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,309
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,309
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,309
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,216
...,...,...,...,...,...,...,...,...,...,...,...,...
880067,2014-12-01,YISHUN,5 ROOM,816,YISHUN ST 81,10 TO 12,122.0,Improved,1988,580000.0,NaN,816
880068,2014-12-01,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,540000.0,NaN,325
880069,2014-12-01,YISHUN,EXECUTIVE,618,YISHUN RING RD,07 TO 09,164.0,Apartment,1992,738000.0,NaN,618
880070,2014-12-01,YISHUN,EXECUTIVE,277,YISHUN ST 22,07 TO 09,152.0,Maisonette,1985,592000.0,NaN,277


In [46]:
# focus on flat_type, flat_model which have redundant titles
def labeler(feat):
    temp = []
    for value in df[feat].unique():
        if value != ' '.join(re.findall( r'\w+', value.upper())):
                df[feat].replace(value,' '.join(re.findall( r'\w+', value.upper())),inplace=True)

In [47]:
for feat in ['flat_type', 'flat_model']:
    labeler(feat)

In [49]:
df['address'] = df['block'] + ' ' + df['street_name'] + ' '+ 'SINGAPORE'
# Get number of unique full addresses
full_address_list = df["address"].unique().tolist()
print(len(full_address_list),full_address_list[1])


9597 216 ANG MO KIO AVE 1 SINGAPORE


In [50]:
df.to_csv('con-1990-2017.csv',index=False)

In [21]:
from geopy.distance import geodesic, great_circle
from geopy.geocoders import GoogleV3
from geopy.geocoders import Nominatim

In [16]:
geolocator = Nominatim(user_agent="GoogleV3")

address = []
full_address = []
latitude = []
longitude = []
no_result = []

for row in range(len(full_address_list)):
    #formulate query string 
    try:
        add, (lat, lon) = geolocator.geocode(full_address_list[row])
        address.append(full_address_list[row])
        full_address.append(add)
        latitude.append(lat)
        longitude.append(lon)
    except:
        no_result.append(full_address_list[row])

df_coordinates  = pd.DataFrame({
    'address': address,
    'full_address': full_address,
    'Latitude': latitude,
    'Longitude': longitude
})

In [18]:
len(df_coordinates)

8410

In [19]:
df_coordinates.to_csv('./locations/hdb_location.csv',index=False)

In [66]:
with open('data//mrt.txt', 'r') as file:
    mrt_list = file.read()

In [67]:
mrt_list=mrt_list.split('\n')

In [68]:
# use the OneMap API to obtain the (lat, long) coordinates of each MRT station.
mrt_building = []
mrt_lat = []
mrt_long = []

for mrt in mrt_list:
    query_address = mrt
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)
    
    if data_mrt['found'] != 0:
        mrt_building.append(data_mrt["results"][0]["BUILDING"])
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_building.append('NotFound')
        mrt_lat.append('NotFound')
        mrt_long.append('NotFound')
        print ("No Results")

# Store this information in a dataframe
mrt_location = pd.DataFrame({
    'MRT': mrt_list,
    'Building': mrt_building,
    'Latitude': mrt_lat,
    'Longitude': mrt_long
})

Jurong East MRT Station,Lat: 1.33315281585758 Long: 103.742286332403
Bukit Batok MRT Station,Lat: 1.34903331201636 Long: 103.749566478309
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39753506936297 Long: 103.747405150236
Kranji MRT Station,Lat: 1.42508698073648 Long: 103.762137459497
Marsiling MRT Station,Lat: 1.43252114855026 Long: 103.774074641403
Woodlands MRT Station,Lat: 1.43605761708128 Long: 103.787938777173
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44905082158502 Long: 103.820046140211
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42944308477331 Long: 103.835005047246
Khatib MRT Station,Lat: 1.41738337009565 Long: 103.832979908243
Yio Chu Kang MRT Station,Lat: 1.38175587099132 Long: 103.84494727118
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.849558

In [69]:
mrt_location

,MRT,Building,Latitude,Longitude
0,Jurong East MRT Station,JURONG EAST MRT STATION (EW24 / NS1),1.33315281585758,103.742286332403
1,Bukit Batok MRT Station,BUKIT BATOK MRT STATION (NS2),1.34903331201636,103.749566478309
2,Bukit Gombak MRT Station,BUKIT GOMBAK MRT STATION (NS3),1.35861159094192,103.751790910733
3,Choa Chu Kang MRT Station,CHOA CHU KANG MRT STATION (NS4),1.38536316540225,103.744370779756
4,Yew Tee MRT Station,YEW TEE MRT STATION (NS5),1.39753506936297,103.747405150236
...,...,...,...,...
115,Bedok North MRT Station,BEDOK NORTH MRT STATION (DT29),1.33474211664091,103.91797832995
116,Bedok Reservoir MRT Station,BEDOK RESERVOIR MRT STATION (DT30),1.33660782955099,103.932234623286
117,Tampines West MRT Station,TAMPINES WEST MRT STATION (DT31),1.34551530560119,103.938436971222
118,Tampines East MRT Station,TAMPINES EAST MRT STATION (DT33),1.35619148271544,103.9546344625


In [70]:
mrt_location.to_csv('./locations/mrt_location.csv',index=False)

In [51]:
# read all the data needed
mrt_location = pd.read_csv('./locations/mrt_location.csv')

# compile multiple location data to one dataframe
df_coordinates = pd.read_csv('./locations/hdb_location.csv')

In [52]:
df_coordinates

,address,full_address,Latitude,Longitude
0,309 ANG MO KIO AVE 1 SINGAPORE,"Ang Mo Kio Avenue 1, Ang Mo Kio, Singapore, Ce...",1.364512,103.842076
1,216 ANG MO KIO AVE 1 SINGAPORE,"216, Ang Mo Kio Avenue 1, Ang Mo Kio Grove, An...",1.366209,103.841476
2,211 ANG MO KIO AVE 3 SINGAPORE,"211, Ang Mo Kio Avenue 3, Kebun Baru, Ang Mo K...",1.369205,103.841719
3,202 ANG MO KIO AVE 3 SINGAPORE,"202, Ang Mo Kio Avenue 3, Ang Mo Kio View, Ang...",1.368461,103.844515
4,235 ANG MO KIO AVE 3 SINGAPORE,"Ang Mo Kio Avenue 3, Kebun Baru, Ang Mo Kio, S...",1.367302,103.835923
...,...,...,...,...
8405,573A WOODLANDS DR 16 SINGAPORE,"573A, Woodlands Drive 16, Woodlands, Singapore...",1.431327,103.798284
8406,693A WOODLANDS AVE 6 SINGAPORE,"Woodlands Avenue 6, Woodlands, Northwest, 7390...",1.433234,103.801529
8407,691A WOODLANDS DR 73 SINGAPORE,"691A, Woodlands Drive 73, Woodlands, Northwest...",1.440301,103.806676
8408,783A WOODLANDS RISE SINGAPORE,"783A, Woodlands Rise, Woodlands Pasture I, Woo...",1.447460,103.804047


In [53]:
address =[]
d_mrt = []
n_mrt = []

for add,lat,lon in zip(df_coordinates.address,df_coordinates.Latitude,df_coordinates.Longitude):
    d_to_mrt = 99
    temp = 0
    
    #loop for MRT and Mall to find the shortestdistance to address
    for mrt,mrt_lat,mrt_lon in zip(mrt_location.Building, mrt_location.Latitude, mrt_location.Longitude):
        temp = great_circle((lat, lon), (mrt_lat, mrt_lon)).km
        if d_to_mrt > temp:
            d_to_mrt = temp
            n_to_mrt = mrt
   
    
    address.append(add)
    d_mrt.append(d_to_mrt)
    n_mrt.append(n_to_mrt)

    
df_coordinates['dist_mrt'] = d_mrt
df_coordinates['near_mrt'] = n_mrt


In [54]:
df_coordinates

,address,full_address,Latitude,Longitude,dist_mrt,near_mrt
0,309 ANG MO KIO AVE 1 SINGAPORE,"Ang Mo Kio Avenue 1, Ang Mo Kio, Singapore, Ce...",1.364512,103.842076,1.027186,ANG MO KIO MRT STATION
1,216 ANG MO KIO AVE 1 SINGAPORE,"216, Ang Mo Kio Avenue 1, Ang Mo Kio Grove, An...",1.366209,103.841476,0.989224,ANG MO KIO MRT STATION
2,211 ANG MO KIO AVE 3 SINGAPORE,"211, Ang Mo Kio Avenue 3, Kebun Baru, Ang Mo K...",1.369205,103.841719,0.875147,ANG MO KIO MRT STATION
3,202 ANG MO KIO AVE 3 SINGAPORE,"202, Ang Mo Kio Avenue 3, Ang Mo Kio View, Ang...",1.368461,103.844515,0.583952,ANG MO KIO MRT STATION
4,235 ANG MO KIO AVE 3 SINGAPORE,"Ang Mo Kio Avenue 3, Kebun Baru, Ang Mo Kio, S...",1.367302,103.835923,1.543686,ANG MO KIO MRT STATION
...,...,...,...,...,...,...
8405,573A WOODLANDS DR 16 SINGAPORE,"573A, Woodlands Drive 16, Woodlands, Singapore...",1.431327,103.798284,1.072885,ADMIRALTY MRT STATION (NS10)
8406,693A WOODLANDS AVE 6 SINGAPORE,"Woodlands Avenue 6, Woodlands, Northwest, 7390...",1.433234,103.801529,0.819941,ADMIRALTY MRT STATION (NS10)
8407,691A WOODLANDS DR 73 SINGAPORE,"691A, Woodlands Drive 73, Woodlands, Northwest...",1.440301,103.806676,0.632860,ADMIRALTY MRT STATION (NS10)
8408,783A WOODLANDS RISE SINGAPORE,"783A, Woodlands Rise, Woodlands Pasture I, Woo...",1.447460,103.804047,0.836188,ADMIRALTY MRT STATION (NS10)


In [55]:
df_coordinates.to_csv('./locations/hdb_location_dist.csv',index=False)

In [56]:
df = pd.read_csv('con-1990-2017.csv')

d:\Program Files\Python\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,block_num,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,216,216 ANG MO KIO AVE 1 SINGAPORE


In [59]:
df_coordinates=df_coordinates.drop_duplicates()

In [60]:
df_final = pd.merge(df,df_coordinates,on=['address'],how='left')

In [61]:
df_final.shape

(880072, 18)

In [64]:
df_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,block_num,address,full_address,Latitude,Longitude,dist_mrt,near_mrt
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE,"Ang Mo Kio Avenue 1, Ang Mo Kio, Singapore, Ce...",1.364512,103.842076,1.027186,ANG MO KIO MRT STATION
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE,"Ang Mo Kio Avenue 1, Ang Mo Kio, Singapore, Ce...",1.364512,103.842076,1.027186,ANG MO KIO MRT STATION
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE,"Ang Mo Kio Avenue 1, Ang Mo Kio, Singapore, Ce...",1.364512,103.842076,1.027186,ANG MO KIO MRT STATION
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,309,309 ANG MO KIO AVE 1 SINGAPORE,"Ang Mo Kio Avenue 1, Ang Mo Kio, Singapore, Ce...",1.364512,103.842076,1.027186,ANG MO KIO MRT STATION
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,216,216 ANG MO KIO AVE 1 SINGAPORE,"216, Ang Mo Kio Avenue 1, Ang Mo Kio Grove, An...",1.366209,103.841476,0.989224,ANG MO KIO MRT STATION


In [66]:
df_final = df_final[['town',
                     'flat_type',
                     'flat_model',
                     'floor_area_sqm',
                     'resale_price',
                     'month',
                     'address',
                     'Latitude',
                     'Longitude',
                     'full_address',
                     'dist_mrt',
                     'near_mrt']]

In [67]:
df_final.to_csv('hdb_resale.csv',index=False)